In [ ]:
import numpy as np
from cobra.io import read_sbml_model

In [ ]:
model = read_sbml_model('')

We want to look at the model. See if it has fluxes

In [ ]:
solution = model.optimize()
solution.fluxes

In [ ]:
model.objective()

In [ ]:
from cobra import Reaction, Metabolite

To make E. Coli produce putrescine 